In [7]:
import pandas as pd
import sklearn as sklearn
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from utils.randomized_search import logUnifD
from utils.base_set import X_train, y_train, seed

### KNN 

In [8]:
model = KNeighborsClassifier()
K = X_train.shape[0]
k = 5
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
param = {
    'n_neighbors': logUnifD(10, K/2),
    'weights': ['uniform', 'distance'],
    'p': stats.randint(1, 4), # minkowski metric
}
n = 100
classifier_KNN = RandomizedSearchCV(estimator=model, 
                                param_distributions=param,
                                cv=cv,
                                n_iter=n,
                                scoring="roc_auc", 
                                random_state=seed)
classifier_KNN.fit(X_train,y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=8241, shuffle=True),
                   estimator=KNeighborsClassifier(), n_iter=100,
                   param_distributions={'n_neighbors': <utils.randomized_search.logUnifD object at 0x73f15b9b3a60>,
                                        'p': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x73f1ac121960>,
                                        'weights': ['uniform', 'distance']},
                   random_state=8241, scoring='roc_auc')

In [9]:
best = classifier_KNN.best_params_
auc_roc = classifier_KNN.best_score_
print(best)
print(auc_roc)

{'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
0.8391199042391625


In [10]:
columns_to_keep = ['param_n_neighbors', 'param_weights', 'param_p', 'mean_test_score','rank_test_score']
pd.DataFrame(classifier_KNN.cv_results_).sort_values('rank_test_score')[columns_to_keep]

,param_n_neighbors,param_weights,param_p,mean_test_score,rank_test_score
6,15,distance,1,0.839120,1
19,11,distance,1,0.838640,2
0,14,distance,1,0.837595,3
64,13,distance,1,0.836097,4
97,26,distance,1,0.828764,5
...,...,...,...,...,...
26,200,distance,3,0.747929,96
93,165,uniform,3,0.742176,97
80,171,uniform,3,0.741821,98
32,198,uniform,3,0.740723,99
